In [1]:
import pandas as pd
import numpy as np
import IPython.display

data = pd.read_csv('./data/filtered_data.csv', usecols=['Id','Title', 'Body', 'Tags'], index_col='Id')
data.reset_index(inplace=True)
data.drop(columns='Id', inplace=True)
data.head(10)

,Title,Body,Tags
0,Can (a== 1 && a ==2 && a==3) ever evaluate to ...,<blockquote>\n <p><strong>Moderator note:</st...,<javascript><ecmascript-6>
1,Git is not working after macOS Update (xcrun: ...,<p>I updated to macOS Mojave (this happens on ...,<xcode><git><macos><command-line><terminal>
2,Why is 2 * (i * i) faster than 2 * i * i in Java?,<p>The following Java program takes on average...,<java><performance><benchmarking><bytecode><jit>
3,Xcode 10 Error: Multiple commands produce,<blockquote>\n <p>error: Multiple commands pr...,<info.plist><xcode10>
4,"Why does array[idx++]+=""a"" increase idx once i...","<p>For a challenge, <a href=""https://stackover...",<java><java-8><javac><java-9><java-10>
5,Difference between npx and npm?,"<p>I have just started learning React, and Fac...",<javascript><npm><npx>
6,Cannot inline bytecode built with JVM target 1...,<p>When trying to run the Example CorDapp (<a ...,<android><intellij-idea><kotlin><jvm><corda>
7,"Could not find module ""@angular-devkit/build-a...","<p>After updating to Angular 6.0.1, I get the ...",<angular><angular-cli><angular6><angular-cli-v6>
8,"How to fix ""ReferenceError: primordials is not...",<p>I have installed Node.js modules by 'npm in...,<node.js><sass><gulp><gulp-sass>
9,Why is the result of ('b'+'a'+ + 'a' + 'a').to...,<p>I was practicing some JavaScript when one o...,<javascript><type-conversion>


## Constitution du corpus 

In [2]:
%%time
data['Post'] = data.apply(lambda x: x['Title'] + ' ' + x['Body'] if x['Title'] == x['Title'] else x['Body'], axis=1)
corpus = data['Post'].to_list()
tags = data['Tags'].to_list()

print("Dix première observations")
display(data.head(10))

print("Premier élément de la liste corpus\n")
display(corpus[0])
print("\n")

print("Premier éléments de la liste tags\n")
display(tags[0])

print(f"Occurences dans le corpus: {len(corpus)}")
print(f"Occurences dans les tags: {len(tags)}")

Dix première observations


,Title,Body,Tags,Post
0,Can (a== 1 && a ==2 && a==3) ever evaluate to ...,<blockquote>\n <p><strong>Moderator note:</st...,<javascript><ecmascript-6>,Can (a== 1 && a ==2 && a==3) ever evaluate to ...
1,Git is not working after macOS Update (xcrun: ...,<p>I updated to macOS Mojave (this happens on ...,<xcode><git><macos><command-line><terminal>,Git is not working after macOS Update (xcrun: ...
2,Why is 2 * (i * i) faster than 2 * i * i in Java?,<p>The following Java program takes on average...,<java><performance><benchmarking><bytecode><jit>,Why is 2 * (i * i) faster than 2 * i * i in Ja...
3,Xcode 10 Error: Multiple commands produce,<blockquote>\n <p>error: Multiple commands pr...,<info.plist><xcode10>,Xcode 10 Error: Multiple commands produce <blo...
4,"Why does array[idx++]+=""a"" increase idx once i...","<p>For a challenge, <a href=""https://stackover...",<java><java-8><javac><java-9><java-10>,"Why does array[idx++]+=""a"" increase idx once i..."
5,Difference between npx and npm?,"<p>I have just started learning React, and Fac...",<javascript><npm><npx>,Difference between npx and npm? <p>I have just...
6,Cannot inline bytecode built with JVM target 1...,<p>When trying to run the Example CorDapp (<a ...,<android><intellij-idea><kotlin><jvm><corda>,Cannot inline bytecode built with JVM target 1...
7,"Could not find module ""@angular-devkit/build-a...","<p>After updating to Angular 6.0.1, I get the ...",<angular><angular-cli><angular6><angular-cli-v6>,"Could not find module ""@angular-devkit/build-a..."
8,"How to fix ""ReferenceError: primordials is not...",<p>I have installed Node.js modules by 'npm in...,<node.js><sass><gulp><gulp-sass>,"How to fix ""ReferenceError: primordials is not..."
9,Why is the result of ('b'+'a'+ + 'a' + 'a').to...,<p>I was practicing some JavaScript when one o...,<javascript><type-conversion>,Why is the result of ('b'+'a'+ + 'a' + 'a').to...


Premier élément de la liste corpus



'Can (a== 1 && a ==2 && a==3) ever evaluate to true? <blockquote>\n  <p><strong>Moderator note:</strong> Please resist the urge to edit the code or remove this notice. The pattern of whitespace may be part of the question and therefore should not be tampered with unnecessarily. If you are in the "whitespace is insignificant" camp, you should be able to accept the code as is.</p>\n</blockquote>\n\n<p>Is it ever possible that <code>(a== 1 &amp;&amp; a ==2 &amp;&amp; a==3)</code> could evaluate to <code>true</code> in JavaScript?</p>\n\n<p>This is an interview question asked by a major tech company. It happened two weeks back, but I\'m still trying to find the answer. I know we never write such code in our day-to-day job, but I\'m curious.</p>\n'



Premier éléments de la liste tags



'<javascript><ecmascript-6>'

Occurences dans le corpus: 46504
Occurences dans les tags: 46504
CPU times: user 488 ms, sys: 23 ms, total: 511 ms
Wall time: 507 ms


## Nettoyage HTML

In [3]:
%%time

def clean_html(text):
    """
    Remove HTML from a text.
    
    Args:
        text(String): Row text with html 
             
    Returns:
        cleaned String
    """
    import lxml
    import html5lib
    from bs4 import BeautifulSoup
 
    soup = BeautifulSoup(text, "html5lib")

    for sent in soup(['style', 'script']):
            sent.decompose()
   
        
    return ' '.join(soup.stripped_strings)

corpus_wo_html = [clean_html(text) for text in corpus]

print("Premier élément de la liste corpus sans html\n")
display(corpus_wo_html[0])
print("\n")

print(f"Occurences dans le corpus: {len(corpus_wo_html)}")

Premier élément de la liste corpus sans html



'Can (a== 1 && a ==2 && a==3) ever evaluate to true? Moderator note: Please resist the urge to edit the code or remove this notice. The pattern of whitespace may be part of the question and therefore should not be tampered with unnecessarily. If you are in the "whitespace is insignificant" camp, you should be able to accept the code as is. Is it ever possible that (a== 1 && a ==2 && a==3) could evaluate to true in JavaScript? This is an interview question asked by a major tech company. It happened two weeks back, but I\'m still trying to find the answer. I know we never write such code in our day-to-day job, but I\'m curious.'



Occurences dans le corpus: 46504
CPU times: user 1min 3s, sys: 35.6 ms, total: 1min 3s
Wall time: 1min 3s


### Nettoyage du texte

In [4]:
%%time

def text_cleaning(text):
    """
    Remove figures, punctuation, words shorter than two letters (excepted C or R) in a lowered text. 
    
    Args:
        text(String): Row text to clean
        
    Returns:
       res(string): Cleaned text
    """
    import re
    
    pattern = re.compile(r'[^\w]|[\d_]')
    
    try: 
        res = re.sub(pattern," ", text).lower()
    except TypeError:
        return text
    
    res = res.split(" ")
    res = list(filter(lambda x: len(x)>3 , res)) #Keep singles c and r because it might be used as name of languages
    res = " ".join(res)
    return res

cleaned_corpus = [text_cleaning(text) for text in corpus_wo_html]
cleaned_tags = [text_cleaning(text).strip() for text in tags]


print("Premier élément de la liste cleaned_corpus\n")
display(cleaned_corpus[0])
print("\n")

print("Premier élément de la liste cleaned_tags\n")
display(cleaned_tags[0])
print("\n")

print(f"Occurences dans le corpus nettoyé: {len(cleaned_corpus)}")
print(f"Occurences dans les tags nettoyés: {len(cleaned_tags)}")

Premier élément de la liste cleaned_corpus



'ever evaluate true moderator note please resist urge edit code remove this notice pattern whitespace part question therefore should tampered with unnecessarily whitespace insignificant camp should able accept code ever possible that could evaluate true javascript this interview question asked major tech company happened weeks back still trying find answer know never write such code curious'



Premier élément de la liste cleaned_tags



'javascript ecmascript'



Occurences dans le corpus nettoyé: 46504
Occurences dans les tags nettoyés: 46504
CPU times: user 6.88 s, sys: 49.8 ms, total: 6.93 s
Wall time: 6.94 s


### Tokenization

In [5]:
%%time
def tokenize(text):
    """
    Tokenize words of a text.
    
    Args:
    
        text(String): Row text
        
    Returns
    
        res(list): Tokenized string.
    """
    
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    from nltk.tokenize import sent_tokenize

    stop_words = set(stopwords.words('english'))
    
    try:
        res = word_tokenize(text, language='english')
    except TypeError:
        return text
    
    res = [token for token in res if token not in stop_words]
    return res

tokenized_corpus = [tokenize(text) for text in cleaned_corpus]
tokenized_tags = [tokenize(text) for text in cleaned_tags]

print("Premier élément de la liste tokenized_corpus\n")
display(tokenized_corpus[0])
print("\n")
print(f"Longueur du premier éléments de liste tokenized_corpus: {len(tokenized_corpus[0])}")
print("\n")

print("Premier élément de la liste tokenized_tags\n")
display(tokenized_tags[0])
print('\n')


print(f"Occurences dans le corpus tokenizé: {len(tokenized_corpus)}")
print(f"Occurences dans la liste des tags: {len(tokenized_tags)}")

Premier élément de la liste tokenized_corpus



['ever',
 'evaluate',
 'true',
 'moderator',
 'note',
 'please',
 'resist',
 'urge',
 'edit',
 'code',
 'remove',
 'notice',
 'pattern',
 'whitespace',
 'part',
 'question',
 'therefore',
 'tampered',
 'unnecessarily',
 'whitespace',
 'insignificant',
 'camp',
 'able',
 'accept',
 'code',
 'ever',
 'possible',
 'could',
 'evaluate',
 'true',
 'javascript',
 'interview',
 'question',
 'asked',
 'major',
 'tech',
 'company',
 'happened',
 'weeks',
 'back',
 'still',
 'trying',
 'find',
 'answer',
 'know',
 'never',
 'write',
 'code',
 'curious']



Longueur du premier éléments de liste tokenized_corpus: 49


Premier élément de la liste tokenized_tags



['javascript', 'ecmascript']



Occurences dans le corpus tokenizé: 46504
Occurences dans la liste des tags: 46504
CPU times: user 31.4 s, sys: 1.33 s, total: 32.7 s
Wall time: 31.6 s


## Filtrage des noms à l'aide d'un modèle de POS tagging

In [6]:
%%time 
def filtering_nouns(tokens):
    """
    Filter singular nouns
    
    Args:
        tokens(list): A list o tokens
        
    
    Returns:
    
        res(list): Filtered token list
    """ 
    
    import nltk
    
    res = nltk.pos_tag(tokens)
    
    res = [token[0] for token in res if token[1] == 'NN']
    
    return res

noun_corpus = [filtering_nouns(tokens) for tokens in tokenized_corpus]

print("Premier élément de la liste noun_corpus\n")
display(noun_corpus[0])
print("\n")

print(f"Longueur du premier éléments de liste noun_corpus: {len(noun_corpus[0])}")
print("\n")


print(f"Occurences dans le corpus tokenizé: {len(noun_corpus)}")

Premier élément de la liste noun_corpus



['moderator',
 'note',
 'please',
 'edit',
 'code',
 'notice',
 'pattern',
 'part',
 'question',
 'whitespace',
 'insignificant',
 'camp',
 'accept',
 'code',
 'javascript',
 'interview',
 'question',
 'tech',
 'company',
 'answer',
 'code']



Longueur du premier éléments de liste noun_corpus: 21


Occurences dans le corpus tokenizé: 46504
CPU times: user 3min 28s, sys: 230 ms, total: 3min 28s
Wall time: 3min 28s


## Lemmatisation

In [21]:
%%time
def lemmatization(tokens):
    """
    Transform tokens into lems 
    
    Args:
        tokens(list): List of tokens
        
    Returns:
        lemmatized(list): List of lemmatized tokens
    """
    import nltk
    from nltk.stem import WordNetLemmatizer
    
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    
    for token in tokens:
        lemmatized.append(lemmatizer.lemmatize(token))
        
    return lemmatized

lemmatized_corpus = [lemmatization(tokens) for tokens in noun_corpus]
lemmatized_tags = [lemmatization(tokens) for tokens in tokenized_tags]

tags_wo_blanks = []
for tokens in lemmatized_tags:
    tokens = [token for token in tokens if len(token)>1]
    tags_wo_blanks.append(tokens)

print("Premier élément de la liste lemmatized_corpus\n")
display(lemmatized_corpus[0])
print("\n")

print(f"Longueur du premier éléments de liste lemmatized_corpus: {len(lemmatized_corpus[0])}")
print("\n")

print("Premier élément de la liste lemmatized_tags\n")
display(tags_wo_blanks[0])


print(f"Occurences dans le corpus lemmatisé: {len(lemmatized_corpus)}")
print(f"Occurences dans les tags lemmatisés: {len(tags_wo_blanks)}")

Premier élément de la liste lemmatized_corpus



['moderator',
 'note',
 'please',
 'edit',
 'code',
 'notice',
 'pattern',
 'part',
 'question',
 'whitespace',
 'insignificant',
 'camp',
 'accept',
 'code',
 'javascript',
 'interview',
 'question',
 'tech',
 'company',
 'answer',
 'code']



Longueur du premier éléments de liste lemmatized_corpus: 21


Premier élément de la liste lemmatized_tags



['javascript', 'ecmascript']

Occurences dans le corpus lemmatisé: 46504
Occurences dans les tags lemmatisés: 46504
CPU times: user 6.18 s, sys: 50 ms, total: 6.23 s
Wall time: 6.22 s


### Filtrage des valeurs vides

In [98]:
joined_corpus = [" ".join(text) for text in lemmatized_corpus]
corpus_df = pd.DataFrame(joined_corpus, columns=['Text'])
corpus_df['len_text'] = corpus_df['Text'].apply(lambda x: len(x))

joined_tags = [" ".join(tags) for tags in tags_wo_blanks]
tag_df = pd.DataFrame(joined_tags, columns=['Tags'])
tag_df['len_tags'] = tag_df['Tags'].apply(lambda x: len(x))

corpus_tag_df = pd.concat([corpus_df, tag_df], axis=1)

empty_data_idx = corpus_tag_df[(corpus_tag_df['len_text']==0) | (corpus_tag_df['len_tags']==0)].index

corpus_tag_df.drop(index=empty_data_idx, inplace=True)
print(f"observations restantes: {corpus_tag_df.shape[0]}")

corpus_tag_df['splitted_text'] = corpus_tag_df['Text'].apply(lambda x: x.split(' ') )
corpus_tag_df['splitted_tags'] = corpus_tag_df['Tags'].apply(lambda x: x.split(' ') )

filtered_corpus = corpus_tag_df['splitted_text'].to_list()
filtered_tags = corpus_tag_df['splitted_tags'].to_list()

corpus_tag_df.to_csv("./data/cleaned_corpus.csv", index=False)

observations restantes: 45902


## Fréquence de distribution tokens

In [45]:
%%time
def build_word_distribution(corpus):
    """
    Build corpus word distribution
    
    Args:
        Corpus(List of lists): Original corpus
    
    Returns:
        
        word_dist_df(DataFrame): Word distribution of the corpus
    """
    
    from nltk import FreqDist
    word_corpus = [token for token_list in lemmatized_corpus for token in token_list]
    word_dist = FreqDist(word_corpus)
    word_dist_df = pd.DataFrame(word_dist.items(), columns=['Word', 'Frequency']).set_index('Word')
    word_dist_df.sort_values("Frequency", ascending=False, inplace=True)

    return word_dist_df

word_dist = build_word_distribution(filtered_corpus)

print(f"Nombre de tokens du corpus {word_dist.shape[0]}")
print("Affichage des 20 tokens les plus utilisés")
display(word_dist.head(20))

Nombre de tokens du corpus 115091
Affichage des 20 tokens les plus utilisés


,Frequency
Word,
java,45874
error,42873
class,33461
name,31880
file,30588
import,27939
code,25988
version,23508
http,22847


CPU times: user 1.12 s, sys: 33 µs, total: 1.12 s
Wall time: 1.12 s


## Fréquence de distribution des tags

In [70]:
tag_dist = build_word_distribution(filtered_tags)
print("Top 20 des tags les plus utilisés")
display(tag_dist.head(20))
print(f"Nombre de tags: {len(tag_dist)}")
first_200_tags = tag_dist[0:200].index

Top 20 des tags les plus utilisés


,Frequency
Word,
java,45874
error,42873
class,33461
name,31880
file,30588
import,27939
code,25988
version,23508
http,22847


Nombre de tags: 115091


In [59]:
tags_wo_blanks[]

['info', 'plist', 'xcode']

### filtrage top 200 tags

In [89]:
filtered_corpus_tag_df = corpus_tag_df.copy()
filtered_corpus_tag_df['tags_in_top200'] = filtered_corpus_tag_df['splitted_tags'].apply(lambda tags: [tag for tag in tags if tag in first_200_tags])

filtered_corpus_tag_df['len_tags_in_top200'] = filtered_corpus_tag_df['tags_in_top200'].apply(lambda x: len(x))
missing_filtered_data = filtered_corpus_tag_df[filtered_corpus_tag_df['len_tags_in_top200'] == 0].index

filtered_corpus_tag_df.drop(index=missing_filtered_data, inplace=True)
print(f"Observations restantes: {filtered_corpus_tag_df.shape[0]}")

top200_corpus = filtered_corpus_tag_df['splitted_text'].to_list()
top200_joined_corpus = filtered_corpus_tag_df['Text'].to_list()
top200_tags = filtered_corpus_tag_df['splitted_tags'].to_list()

Observations restantes: 33169


### Fréquences de distribution des tokens après filtrage du top 200 tags

In [90]:
word_dist_top200 = build_word_distribution(top200_corpus)

print(f"Nombre de tokens du corpus {word_dist_top200.shape[0]}")
print("Affichage des 20 tokens les plus utilisés")
display(word_dist_top200.head(20))
word_dist_top200[word_dist_top200["Frequency"]>=1000].shape[0]

Nombre de tokens du corpus 115091
Affichage des 20 tokens les plus utilisés


,Frequency
Word,
java,45874
error,42873
class,33461
name,31880
file,30588
import,27939
code,25988
version,23508
http,22847


476

### Fréquences de distribution des tags après filtrage du top 200 tags

In [91]:
tag_dist_top_200 = build_word_distribution(top200_tags)
print("Top 20 des tags les plus utilisés")
display(tag_dist_top_200.head(20))
print(f"Nombre de tags: {len(tag_dist_top_200)}")

Top 20 des tags les plus utilisés


,Frequency
Word,
java,45874
error,42873
class,33461
name,31880
file,30588
import,27939
code,25988
version,23508
http,22847


Nombre de tags: 115091


### TF-IDF

In [95]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vocabulary = list(word_dist_top200[word_dist_top200["Frequency"]>=1000].index)
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulary)
X = tfidf_vectorizer.fit_transform(top200_joined_corpus)
tfidf_data = pd.DataFrame(X.toarray(), columns=vocabulary)
tfidf_data
tfidf_data.shape[0]

CPU times: user 902 ms, sys: 80.4 ms, total: 982 ms
Wall time: 980 ms


33169

### Partition des données

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, top200_tags, test_size=0.2, random_state=42)

In [97]:
y_test

[['javascript', 'html', 'scrollspy'],
 ['python', 'kera'],
 ['javascript', 'python', 'reactjs', 'flask', 'flask', 'restful'],
 ['core', 'core', 'smtp', 'mailkit'],
 ['html', 'form', 'input', 'autocomplete', 'material'],
 ['node', 'typescript'],
 ['blazor', 'blazor', 'server', 'side', 'core', 'core'],
 ['java', 'android'],
 ['python', 'python'],
 ['entity', 'framework', 'core'],
 ['amazon', 'cloudformation', 'parameter', 'store'],
 ['dart', 'flutter', 'widget', 'material'],
 ['android',
  'kotlin',
  'kotlin',
  'android',
  'extension',
  'kotlin',
  'extension'],
 ['python', 'selenium', 'selenium', 'webdriver', 'captcha', 'simplecaptcha'],
 ['android',
  'sqlite',
  'android',
  'room',
  'sqliteopenhelper',
  'android',
  'architecture',
  'component'],
 ['module'],
 ['python',
  'visual',
  'studio',
  'code',
  'python',
  'import',
  'vscode',
  'setting',
  'pylance'],
 ['flutter', 'dart', 'text'],
 ['angular', 'request', 'observable'],
 ['docker', 'virtualization', 'qemu', 'virs